# Magentic-One

Magentic-One は Microsoft Research が開発した汎用マルチエージェントシステムです。
Magentic-One の `Orchestrator` エージェントは、計画を作成し、他のエージェントにタスクを委任し、目標に向けた進捗状況を追跡して、必要に応じて計画を動的に修正します。`Orchestrator` は、ファイルの読み取りと処理を行う `FileSurfer` エージェント、Web ブラウザーを操作する WebSurfer エージェント、またはコードの記述や実行を行う `Coder` エージェントまたは `Computer Terminal` エージェントにタスクを委任できます。

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/magentic-one.html

<img src="https://microsoft.github.io/autogen/stable/_images/autogen-magentic-one-example.png" width="800"/>

## 内部処理解説
![](./img/004.png)

https://www.youtube.com/watch?feature=shared&t=5408&v=jHW6zcIzm7g

### 1. Outer Loop（外側のループ）:
タスクの初期計画を作成し、進行状況を管理する役割を持ちます。タスク台帳 (Task Ledger) を維持し、与えられた情報、検証済みの事実、未調査の事実、推測情報を格納します。

さらに各エージェントの役割を考慮しながら、タスクを分解し、作業計画を立案します。計画が更新されるたびに、すべてのエージェントの状態をリセットします。

### 2. Inner Loop（内側のループ）
タスク進行中に、エージェントの動作を調整し、失敗時の対応を決定します。進捗台帳 (Progress Ledger) を維持し、作業の進行状況、繰り返し発生している問題、どのエージェントが次に動作するかを記録します。

## Installing Playwright
事前に Node.js のインストールが必要です。

https://playwright.dev/docs/intro

In [ ]:
#!npm init playwright@latest

#!pip install "autogen-ext[magentic-one]"
#!playwright install --with-deps chromium

In [ ]:
import logging  
from typing import Any, List
import os
from dotenv import load_dotenv  

from autogen_agentchat.agents import AssistantAgent  
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.agents.magentic_one import MagenticOneCoderAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.teams import MagenticOneGroupChat  # keeps implementation simple & familiar  
from autogen_agentchat.conditions import TextMessageTermination, MaxMessageTermination, TextMentionTermination, TimeoutTermination

from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor

from autogen_core import CancellationToken  
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  
import warnings
warnings.simplefilter('ignore')

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## ツール定義(MCP via SSE)

In [ ]:

server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Number of Tools: {len(tools)}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## エージェント定義

In [ ]:

# 3. -----------------  Agent Definitions -----------------  
analysis_planning_agent = AssistantAgent(  
    name="AnalysisPlanningAgent",  
    model_client=model_client,  
    description="タスクを計画するエージェント。新しいタスクが与えられたときに最初に起動するエージェントであるべきである。",
    system_message=(  
"""
あなたは「分析 & 計画エージェント（Analysis & Planning Agent）」です – 全体のオーケストレーターとして機能します。

あなたの役割:
1) 顧客からの抽象的なリクエストを解析すること。
2) リクエストを明確なサブタスクに分解し、それぞれを専門エージェント
   （crm_billing、product_promotions）に割り当てること。
3) 各専門エージェントからの出力を統合し、1つの包括的で一貫性のある
   顧客向けの回答としてまとめること。
4) 満足のいく結果が得られたら、以下のプレフィックスを付けて顧客に最終回答すること
   TERMINATE

調査すべき内容をすべて調査し終わったら調査結果を要約し、文の最後に TERMINATE を含めること!
プランだけ立てて TERMINATE するのは禁止です。

"""
)
)  

crm_billing_agent = AssistantAgent(  
    name="CRMBillingAgent",  
    model_client=model_client,  
    description="CRM & 請求エージェントのエージェント。CRM／請求システム、SNS(Tweet) データベースを照会する",
    tools=tools,  
    system_message=(
"""
あなたは「CRM & 請求エージェント（CRM & Billing Agent）」です。

- アカウント、サブスクリプション、請求書、支払い情報などを取得するために、
  構造化されたCRM／請求システムを照会します。
- SNS(ツイート) データは登録してある Tools を使用して取得できます。
- 請求ポリシー、支払い処理、返金ルールなどに関する *ナレッジベース* 記事を確認し、
  回答が正確かつポリシーに準拠していることを保証します。
- 簡潔で構造化された情報を返答し、検出されたポリシー上の懸念事項があれば
  フラグを立てます。
"""
    ),  
)  

product_promotions_agent = AssistantAgent(  
    name="ProductPromotionsAgent",  
    model_client=model_client,  
    description="製品 & プロモーションエージェント。プロモーションのオファー、製品の在庫状況、適格条件、割引情報などを照会",
    tools=tools,  
    system_message=(  
"""
あなたは「製品 & プロモーションエージェント（Product & Promotions Agent）」です。

- 構造化された情報源から、プロモーションのオファー、製品の在庫状況、
  適格条件、割引情報などを取得します。
- *ナレッジベース* のFAQ、利用規約、ベストプラクティスを活用して、
  回答を補足します。
- 事実に基づいた、最新の製品／プロモーション情報を提供します。
"""
    ),  
)  

surfer_agent = MultimodalWebSurfer(
    "WebSurfer",
    model_client=model_client,
    description="Webサーフィンを行うエージェント。Webページを検索し、情報を取得することができます。",
    to_save_screenshots=True,
    browser_data_dir="./browser_data",
    debug_dir="./debug",
    downloads_folder="./downloads",
)

corder_agent = MagenticOneCoderAgent(
    "Corder",
    description="コードを生成するエージェント。与えられたプロンプトに基づいてコードを生成します。",
    model_client=model_client
    )

executor_agent = CodeExecutorAgent("ComputerTerminal", code_executor=LocalCommandLineCodeExecutor())

In [ ]:
# 4. -----------------  Assemble Team -----------------  
participants: List[AssistantAgent] = [  
    analysis_planning_agent,  
    crm_billing_agent,
    product_promotions_agent,
    surfer_agent,
    corder_agent,
    executor_agent
]  

## 停止条件
AutoGen には 無限ループを防止するため 8 つの組み込みの終了条件が定義されています。終了条件は以下のように OR 条件で指定できるのが便利です。



In [ ]:
# Define termination condition
max_msg_termination = MaxMessageTermination(max_messages=25)
text_termination = TextMentionTermination("TERMINATE")
time_terminarion = TimeoutTermination(240)
combined_termination = max_msg_termination | text_termination | time_terminarion

## Magentic-One の実行

In [ ]:
with tracer.start_as_current_span("Magentic-One") as rollspan:

    team = MagenticOneGroupChat(participants=participants, model_client=model_client, termination_condition=combined_termination, max_turns=10)
    task = "Hanagasaki の文字列をMD5 でハッシュ化してください。"
    await Console(team.run_stream(task=task))

`Coder` および `ComputerTerminal` が実行されたことを確認してください。

In [ ]:
with tracer.start_as_current_span("Magentic-One") as rollspan:

    team = MagenticOneGroupChat(participants=participants, model_client=model_client, termination_condition=combined_termination, max_turns=20)
    task = "草津温泉でいくべき観光スポットと最新のイベント情報を教えてください。"
    await Console(team.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Magentic-One") as rollspan:

    team = MagenticOneGroupChat(participants=participants, model_client=model_client, termination_condition=combined_termination)
    task = "2024年9月~10月の受注数を集計し、さらにSNS分析を行い、日付ごとレコードを集計した結果何かわかることはあるか？"
    await Console(team.run_stream(task=task))